## Class Diagram
<img src="class_diagram.jpg" alt="" />

Import the *Simulator* module and *Pulse* module for simulation and *Optimizer* module for optimization.

In [1]:
from Simulator import *
from Pulse import *

In [ ]:
from Optimizer import *

### Global variables

In [ ]:
#The Paulis
X = np.array([[0,1],[1,0]], dtype=np.complex128)
Y = np.array([[0, -1j],[1j,0]], dtype=np.complex128)
Z = np.array([[1,0],[0,-1]], dtype=np.complex128)
I = np.array([[1,0],[0,1]], dtype=np.complex128)
   
#Pauli eigenvectors
Xp = 0.5*np.array([[1.,1.],[1.,1.]], dtype=np.complex128) #X+
Xm = 0.5*np.array([[1.,-1.],[-1.,1.]], dtype=np.complex128) #X-
Yp = 0.5*np.array([[1.,-1j],[1j,1.]], dtype=np.complex128) #Y+
Ym = 0.5*np.array([[1.,1j],[-1j,1.]], dtype=np.complex128) #Y-
Zp = np.array([[1.0,0.0],[0.0,0.0]], dtype = np.complex128) #Z+
Zm = np.array([[0.0,0.0],[0.0,1.0]], dtype = np.complex128) #Z-

rho = np.array([Xp, Xm, Yp, Ym, Zp, Zm])
O = np.array([X, Y, Z])
zero = np.array([[1,0],[0,0]], dtype = np.complex128)

Gate_names= ["I", "X", "Y", "Z", "H", "pi4"] 
Gate = [np.array([[1.,0.],[0.,1.]]), np.array([[0.,1.],[1.,0.]]), np.array([[0.,-1j],[1j,0.]]), np.array([[1.,0],[0.,-1.]]), np.array([[1.,1.],[1.,-1.]])/np.sqrt(2),expm(-1j*np.pi*0.25*np.array([[0.,1.],[1.,0.]])/2) ]



### Pulse

*Pulse(T, name, params, T, t_step, time_range=[])*

This is the class to generate pulses.

The parameters:
1. name: string, pulse type (Instantaneous, Gaussian, CPMG, CPMG_XY)
2. params: dictionary, parameters of the pulse
3. T: float, time T
4. num_steps: integer, number of discrete time steps
5. time_range: list, default = []

Class attributes:
1. *name*: string
2. *params*: objdict
    - Instantaneous:
        - params.num_pulses: integer, number of pulses
        - params.A_max: float, default = 2
        - params.theta: float, default = None
        - params.vec: list, default = None
        - params.axes: integer, default = 3
            - axes = 1 : only in X-axis
            - axes = 2 : X and Y axis
            - axes = 3 : X, Y, and Z axis
        - params.position: list, default = None
    - Gaussian:
        - params.num_pulses: integer
        - params.N: integer, the number of datasets
        - params.amp_scale: integer, default = 1
        - params.amplitude: np.array([axes, num_pulses])
        - params.position: list
        - params.sd: float, the standard deviation
        - params.axes: integer, default = 1
    - CPMG:
        - params.num_pulses: integer
        - params.A_max: integer, default = 1
    - CPMG_XY:
        - params.num_pulses: integer
    - Zero:
        - params.axes: integer, default = 3
3. *T*: float
4. *num_pulses*: integer, number of discrete time steps
5. *time_range*: list
6. *pulses*: np.array()
    - Instantaneous: shape = (1,4,4,...,4)
    - Gaussian: shape = (N,M,axes)
    - CPMG: shape = (1,M,1)
    - CPMG_XY: shape = (1,M,2)
    - Zero: shape = (1,M,axes)
7. *pulses_params*: np.array()
    - Instantaneous: shape = (num_pulses, axes)
    - Gaussian: shape = (N, num_pulses, 2*axes), (amplitude_axis, position_axis)

The methods:
1. *generate_pulse()*: To generate pulses, this method works for any pulses.
2. *convert_to_gaussian(position=None, sd=None)*: Convert Instantaneous pulses to Gaussian pulses


In [ ]:
p = Pulse(name = "Instantaneous", params={"n_max":2}, T=1, M=1000)
p.generate_pulse()
pulses = p.pulses

### Quantum Model

*QuantumModel(operators, constants, noise_parameters, num_qubits =1, num_aux = 0, time_range=[])*

The *QuantumModel* class is the model of the system described by the system's Hamiltonian.

\begin{equation}
H = H_{d} + H_{control} + H_{noise}
\end{equation}

The parameters:
1. **operators** : dictionary of operators in the Hamiltonian \
    a. *drift* : list of drift operators decribing $H_d$\
    b. *control* : list of control operators decribing $H_{control}$\
    c. *noise* : list of noise operators \
    d. *collapse* : list \
    e. *measurement* : list 
2. **constants** : dictionary of constants\
    a. *drift* : list of constants for static operators\
    b. *collapse* : list of constants for collapse operators\
    c. *T* : float, time T\
    d. *num_steps* : integer, number of discrete time steps\
    e. *num_realizations* : integer, number of realizations\
    f. *g* : list of float (len operators.noise), quantum noise coupling strength\
    g. *pulse_positions*: list of floats, the positions of pulses
3. **noise_parameters** : dictionary of noise parameters\
    a. *noise_type* : string, type of noise (RTN, RTN_mod, Quasi_static, Gaussian, Quantum, SC_Noise)\
        - RTN -> gamma\
        - RTN_mod -> gamma, Omega_noise\
        - Quasi_static -> None\
        - Gaussian -> sd\
        - Quantum -> None\
        - SC_Noise -> alpha\
    b. *constants* : dictionary of noise constants (this will depend on the noise type)\
        - gamma : list of float, transition rate of RTN process\
        - sd : list of float, standard deviation for Gaussian noise\
        - Omega_mod : list of float, modulation frequency of RTN mod process
        - alpha : list of float
4. **num_qubits** : integer, number of qubits in the system, default = 1
5. **num_aux** : integer, number of qubits in the environment, default = 0
6. **time_range** : list, default = []

Class attributes:
1. *operators* : objdict
    - operators.drift : list
    - operators.control : list
    - operators.lindblad : list
    - operators.measurement : list
2. *constants* : objdict
    - constants.drift : list
    - constants.lindblad : list
    - constants.T : float
    - constants.num_step : integer
    - constants.num_realizations : integer
    - constants.g : list of float
    - constants.instantaneous_pulse_position : list
3. *time_range* : np.array
4. *noise_parameters* : objdict
    - noise_parameters.noise_type : string
    - noise_parameters.constants : objdict
        - noise.parameters.constants.gamma : list of float
        - noise.parameters.constants.sd : list of float
        - noise.parameters.constants.Omega_mod : list of float
5. *num_qubits* : integer
6. *num_aux* : integer
7. *dim* : integer, dimension of the system+auxiliary ($2^{\text{\#qubits\_sys +\#qubits\_aux}}$)
8. *dim_sys*: integer, dimension of the system
9. *noise* : np.array([num_realizations,num_steps,#noise_operators])
10. *V_O* : np.array(1,4,4,...,4,2,2)
11. *num_pulses* : number of instantaneous pulses if constants.instantaneous_pulse_position is defined

Methods that can be accessed of this class:
1. *save_Qmodel(file_name)* : save the instantiation of QuantumModel class to a file
2. *load_Qmodel(file_name)* : load the class object from a file
3. *generate_noise()* : generate the noise, noise is stored in the attribute *noise*. Noise is generated automatically when constructing the object.
4. *generate_V_O(batch_size)*: generate $V_O$ matrices, $V_O$ is stored in the attribute *V_O*

To create an object, call the *QuantumModel* class as follows and pass all the required parameters.

In [ ]:
operators = {"static": [Z], "dynamic": [X,Y], "noise": [Z]}
noise = {"noise_type":"RTN_mod", "constants":{"gamma":0.01, "Omega_mod":[0]}}
constants = {"drift": [10], "T":1, "num_steps":1000, "num_realizations":1000, "g":[0]}
Qmodel = QuantumModel(operators = operators, constants = constants, noise_parameters = noise)

### Simulator

*Simulator(Qmodel, sim_type, init_states=None, observable=None)*

The *Simulator* class takes the following parameters:
1. **Qmodel** : QuantumModel
2. **sim_type** : string, simulator type.
    The types of simulator are:
    - *Choi* : compute the process fidelity
    - *State_t* : compute final states for all time *t*
    - *State_T* : compute final states at time *T*
    - *Observable_t* : compute expectations for all time *t*
    - *Observable_T* : compute expectations at time *T*
    - *State_Observable_t* : compute both final states and expectations for all time *t*
    - *State_Observable_T* : compute both final states and expectations at time *T*
3. **init_states** : list of initial states
4. **observables** : list of observables

Class attributes:
1. *rho_0* : np.array, default = $rho \otimes \left| 0 \rangle\langle 0 \right|^{\otimes \text{num\_aux}}$
2. *Obs* : np. array, default = $O \otimes I^{\otimes \text{num\_aux}}$

To create a simulator, call the class *Simulator* as the following.

In [ ]:
sim = Simulator(Qmodel = Qmodel, sim_type = "Observable_T") 

Methods that can be accessed:
1. *process_fidelity(G, pulses, batch_size=1)* : returns the process fidelity
    - G : np.array, the target gate
    - pulses : np.array([num_realizations,num_steps,pulse_dim])
    - batch_size : integer
2. *simulate(pulses, batch_size=1)* : simulate the quantum evolution
    - pulses : np.array([num_realizations,num_steps,pulse_dim])
    - batch_size : integer
3. *get_rho_t()* : returns final states for all *t* of shape np.array([1,#initial states,M,state dim,state dim])
4. *get_rho_T()* : returns final states at time T of shape np.array([1, #initial states, state dim, state dim])
5. *get_obs_t()* : returns expectations for all time t of shape np.array([1,num_steps,#observable])
6. *get_obs_T()* : returns all expectations at time T of shape np.array([1,#observable])

In [ ]:
sim.simulate(pulses = pulses, batch_size = 100)
rho_t = sim.get_rho_t()

### Optimizer

*Optimizer(Qmodel, optimizer_params, A_max = 1)*

The parameters of this class are:
1. **Qmodel**: QuantumModel
2. **optimizer_params**: dictionary of optimizer parameters\
    a. *learning_rate* : float, learning rate of the optimizer \
    b. *loss* : string, loss function\
    c. *type* : string, Choi or Observable
3. **num_pulses** : integer, default = 5, number of pulses
4. **A_max** : float, default = 1, multiplier of pi-pulse

Class attributes:
1. *Qmodel*: QuantumModel
2. *optimizer_params*: dictionary
3. *num_pulses*: integer
4. *A_max*: float
5. *axes*: integer
6. *learning_rate*: float
7. *loss*: function
8. *pulses*: np.array(), shape:
    - Instantaneous: [1,4,4,4,...,4]
    - Others: [1,num_realizations,axes]
9. *pulses_params*: np.array(), shape:
    - Instantaneous:[num_pulses,axes]
    - Others: [1,num_pulses,2*axes]

Methods that can be accessed:
1. *optimize(G, num_iterations, batch_size)* : compute the optimized control pulses
    - G : np.array, the target gate
    - num_iterations: integer
    - batch_size : integer
        

In [ ]:
opt = Optimizer(Qmodel = Qmodel, optimizer_params = {"learning_rate": 0.01, "loss": "matrix_norm", "type": "Observable"})

### Performance

*Performance(Qmodel=None)*

The *Performance* class takes *QuantumModel* object as the parameter.

Methods that can be accessed:
1. *Optimization_history(opt_history)* : plot the optimization history
    - opt_history: list

2. *fidelity(G, model, noise, batch_size)* : returns process fidelity
    - G: np.array, the target gate
    - model: keras model
    - noise: np.array([K,M,1]), the system noise
    - batch_size: integer

3. *plot_states(states, elt_idx, state_idx)* : plot the the *elt_idx* element of states of index *state_idx*
    - states: np.array, array of states
    - elt_idx: list of 2 elements
    - state_idx: integer

4. *plot_pulses(pulses, k)* : plot pulses of k-th realization
    - pulses: np.array([K,M,n_pulses])
    - k: integer

5. *plot_coherence(states)* : plot the coherence
    - states: np.array, array of states

6. *plot_purity(states)* : plot the purity
    - states: np.array, array of states
    

In [ ]:
from Performance import *

pf = Performance(Qmodel)
pf.plot_coherence(states)